# Rheostat Attempt 6 - Parameter Search
In this notebook, we will attempt to perform a parameter search to better understand the dynamics and attempt to choose an enzymatic model! <br>
5/4/2020

In [1]:
# Basic Imports

#A Model is a CRN with some bells and whistles
from bioscrape.types import Model

#py_simulate_model is a helper function that takes care of may details for you
from bioscrape.simulator import py_simulate_model

#For arrays and plotting
import numpy as np
#import pylab as plt

# Import good plotting packages 
import bokeh.io
import bokeh.plotting
from bokeh.layouts import row
from bokeh.layouts import column
#import bokeh_catplot

import panel as pn

import warnings
warnings.filterwarnings("ignore")

bokeh.io.output_notebook()
pn.extension()

Loading BokehJS ...

## Parameter reasoning
In general, we will model enzyme reactions such as: $$F + S + E \leftrightarrow^{k_{bf}}_{k_{br}} F:S:E \rightarrow^{K_{cat}} W:P:E \leftrightarrow^{k_{uf}}_{k_{ur}} \rightarrow W + P + E$$ <br>
Now, we will attempt to get valid parameter values for concentrations and rate constants. <br>
**Concentrations** <br>
We will denote everything in mM. It may be important to distinguish between steady state and initial concentrations.
- [ATP] - 5 mM in E.Coli 
   - used 0.7 mM in Opgenorth et al. paper
- [ADP] - 0.5 mM in E.Coli
- [Pi] - 10 mM in E.Coli
    - (PBOC2 pg276)
    - these are steady state concentrations, not initial concentrations
- [ATP] - 1.5 mM
    - concentration of initial ATP used in TX/TL extract
- [Glucose] - 1.5 mM
    - Bionum ID 109804
    - In de-repressed yeast cells
    - 400 mM in Opgenorth et al. paper
- [Glucose] - 30 mM
    - concentration of 3-PGA extract used in TX/TL
- [Enzyme1], [Enzyme2], [Enzyme3]
    - based on page 597 of PBOC2, we should let the initial concentrations of enzymes in the cell be equal to the concentration of substrate, so glucose or ATP in this case
    
- [All Other Enzymes/ Enzyme4]
    - can be thought of as ATPase
    - maybe around the same concentration as the other enzymes
  
**Rate Constants** <br>
- $k_{bf}$ of ATP Synthase - $3.6*10^{-7} M^{-1} s^{-1}$
    - bionum ID 104892
    - Considering that we are inputting 1.5 mM of ATP, this number becomes 0.378 $s^{-1}$
- $k_{uf}$ - $0.04 s^{-1}$
    - bionum id 109772
- $k_{cat}$ - $200 s^{-1}$
    - bionum id 100354
We can reason some values for $k_{br}$ and $k_{ur}$. We would expect that they are at least 10 orders of magnitude below the rate constants for the forward version.
- $k_{br} \sim 0.0378 s^{-1}$
- $k_{ur} \sim 0.004 s^{-1}$

 

## Parameter Search

First, we will define three functions (model1_df, model2_df, and model3_df) that will model each enzymatic model. Each function will output 2 dataframes - one with the comprehensive pathway and one that only models ATP leak.

In [2]:
def model1_df(parameters, x_0, x_0_atp_leak, t_max):
    
    # 1 step model
    species = ['atp', 'glucose', 'e1', '2atp:glucose:e1', '2adp:f16p:e1', 'adp', 'pi', 'f16p',
           'e2', '2adp:f16p:e2','2atp:isobutanol:e2', 
           '3adppi:f16p:e3', 'e3' , '3atp:isobutanol:e3', 'isobutanol',
          'atp:e4', 'adppi:e4', 'e4']

    # reaction 1
    rxn1_f = (['atp', 'atp', 'glucose', 'e1'], ['2atp:glucose:e1'],
               'massaction',  {'k': 'k_bf'})

    rxn1_r = ( ['2atp:glucose:e1'],['atp', 'atp', 'glucose', 'e1'], 
                'massaction', {'k': 'k_br'})

    # reaction 2
    rxn2_f = ( ['2atp:glucose:e1'],['2adp:f16p:e1'], 
               'massaction', {'k': 'k_cat'})

    # reaction 3
    rxn3_f =  ( ['2adp:f16p:e1'], ['adp', 'adp', 'f16p', 'e1'], 
               'massaction', {'k': 'k_uf'})

    rxn3_r =  (['adp', 'adp', 'f16p', 'e1'], ['2adp:f16p:e1'],
               'massaction', {'k': 'k_ur'})

    # reaction 4
    rxn4_f =  (['f16p', 'adp', 'adp', 'e2'], ['2adp:f16p:e2'],
               'massaction', {'k': 'k_bf'})

    rxn4_r =  ( ['2adp:f16p:e2'], ['f16p', 'adp', 'adp', 'e2'],
               'massaction', {'k': 'k_br'})

    # reaction 5 
    rxn5_f =  (['2adp:f16p:e2'], ['2atp:isobutanol:e2'],
               'massaction', {'k': 'k_cat'})

    # reaction 6
    rxn6_f =  (['2atp:isobutanol:e2'], ['atp', 'atp', 'isobutanol', 'e2'],
               'massaction', {'k': 'k_uf'})

    rxn6_r =  (['atp', 'atp', 'isobutanol', 'e2'], ['2atp:isobutanol:e2'],
               'massaction', {'k': 'k_ur'})

    # reaction 7
    rxn7_f = ( ['adp', 'adp', 'adp', 'pi', 'f16p', 'e3'],['3adppi:f16p:e3'], 
                  'massaction', {'k': 'k_bf'})

    rxn7_r = ( ['3adppi:f16p:e3'],['adp', 'adp', 'adp', 'pi', 'f16p', 'e3'], 
                  'massaction', {'k': 'k_br'})

    # reaction 8
    rxn8_f = ( ['3adppi:f16p:e3'],['3atp:isobutanol:e3'], 
                  'massaction', {'k': 'k_cat'})

    # reaction 9
    rxn9_f = ( ['3atp:isobutanol:e3'],['atp', 'atp','atp',  'isobutanol', 'e3'], 
                  'massaction', {'k': 'k_uf'})

    rxn9_r = ( ['atp', 'atp', 'atp', 'isobutanol', 'e3'],['3atp:isobutanol:e3'], 
                  'massaction', {'k': 'k_ur'})

    # ATP leak
    rxn10_f = (["atp", 'e4'], ['atp:e4'], "massaction", {"k":"k_bf"})
    rxn10_r = (['atp:e4'], ["atp", 'e4'],  "massaction", {"k":"k_br"})

    rxn11_f = (['atp:e4'], ["adppi:e4"],  "massaction", {"k":"k_cat"})

    rxn12_f = ( ["adppi:e4"], ['adp', 'pi', 'e4'], "massaction", {"k":"k_uf"})
    rxn12_r = (['adp', 'pi', 'e4'], ["adppi:e4"] , "massaction", {"k":"k_ur"})


    reactions = [rxn1_f, rxn1_r, rxn2_f, rxn3_f, rxn3_r,rxn4_f,
                 rxn4_r, rxn5_f, rxn6_f,  rxn6_r, rxn7_f,
                 rxn7_r, rxn8_f, rxn9_f, rxn9_r, 
                 rxn10_f, rxn10_r, rxn11_f, rxn12_f, rxn12_r]

    M = Model(species = species, reactions = reactions, parameters = parameters, initial_condition_dict = x_0)
    timepoints = np.linspace(0,t_max,1000)
    df_results = py_simulate_model(timepoints, Model = M, stochastic = False)

    # Model for only the atp leak
    species_atp_leak = ['atp', 'adp', 'pi', 'e4', 'atp:e4', 'adppi:e4']
    reactions_atp_leak = [rxn10_f, rxn10_r, rxn11_f, rxn12_f, rxn12_r]
    M_atp_leak = Model(species = species_atp_leak, reactions = reactions_atp_leak, parameters = parameters,
                      initial_condition_dict = x_0_atp_leak)
    df_results_atp_leak = py_simulate_model(timepoints, Model = M_atp_leak, stochastic = False)
    
    return [df_results, df_results_atp_leak]

In [3]:
def model2_df(parameters, x_0, x_0_atp_leak, t_max):
    
    # 3 step model v1
    species = ['glucose', 'e1', 'atp', 'glucose:e1', '2atp:glucose:e1', '2adp:f16p:e1', 'adp', 'pi', 'f16p:e1', 'f16p',
              'e2', 'f16p:e2', '2adp:f16p:e2', '2atp:isobutanol:e2', 'isobutanol:e2',
              'e3', 'f16p:e3', '3adppi:f16p:e3', '3atp:isobutanol:e3', 'isobutanol:e3', 'isobutanol',
              'e4', 'atp:e4', 'adppi:e4']


    rxn1_f = (['glucose', 'e1'], ['glucose:e1'], 'massaction', {'k': 'k_bf'})
    rxn1_r = (['glucose:e1'], ['glucose', 'e1'], 'massaction', {'k': 'k_br'})

    rxn2_f = (['atp', 'atp', 'glucose:e1'], ['2atp:glucose:e1'], 'massaction', {'k': 'k_bf'})
    rxn2_r = (['2atp:glucose:e1'], ['atp', 'atp', 'glucose:e1'], 'massaction', {'k': 'k_br'})

    rxn3_f = (['2atp:glucose:e1'], ['2adp:f16p:e1'], 'massaction', {'k': 'k_cat'})

    rxn4_f = (['2adp:f16p:e1'], ['adp', 'adp', 'f16p:e1'], 'massaction', {'k': 'k_uf'})
    rxn4_r = (['adp', 'adp', 'f16p:e1'], ['2adp:f16p:e1'], 'massaction', {'k': 'k_ur'})

    rxn5_f = (['f16p:e1'], ['f16p', 'e1'], 'massaction', {'k': 'k_uf'})
    rxn5_r = (['f16p', 'e1'], ['f16p:e1'], 'massaction', {'k': 'k_ur'})

    rxn6_f = (['f16p', 'e2'], ['f16p:e2'], 'massaction', {'k': 'k_bf'})
    rxn6_r = (['f16p:e2'], ['f16p', 'e2'], 'massaction', {'k': 'k_br'})

    rxn7_f = (['adp', 'adp','f16p:e2'], ['2adp:f16p:e2'], 'massaction', {'k': 'k_bf'})
    rxn7_r = (['2adp:f16p:e2'], ['adp', 'adp', 'f16p:e2'], 'massaction', {'k': 'k_br'})

    rxn8_f = (['2adp:f16p:e2'], ['2atp:isobutanol:e2'], 'massaction', {'k': 'k_cat'})

    rxn9_f = (['2atp:isobutanol:e2'], ['atp', 'atp', 'isobutanol:e2'], 'massaction', {'k': 'k_uf'})
    rxn9_r = (['atp', 'atp', 'isobutanol:e2'], ['2atp:isobutanol:e2'], 'massaction', {'k': 'k_ur'})

    rxn10_f = (['isobutanol:e2'], ['isobutanol', 'e2'], 'massaction', {'k': 'k_uf'})
    rxn10_r = (['isobutanol', 'e2'], ['isobutanol:e2'], 'massaction', {'k': 'k_ur'})

    rxn11_f = (['f16p', 'e3'], ['f16p:e3'], 'massaction', {'k': 'k_bf'})
    rxn11_r = ( ['f16p:e3'], ['f16p', 'e3'], 'massaction', {'k': 'k_br'})

    rxn12_f = (['adp', 'adp', 'adp', 'pi', 'f16p:e3'], ['3adppi:f16p:e3'], 'massaction', {'k': 'k_bf'})
    rxn12_r = (['3adppi:f16p:e3'], ['adp', 'adp', 'adp', 'pi', 'f16p:e3'], 'massaction', {'k': 'k_br'})

    rxn13_f = (['3adppi:f16p:e3'], ['3atp:isobutanol:e3'], 'massaction', {'k': 'k_cat'})

    rxn14_f = (['3atp:isobutanol:e3'], ['atp', 'atp', 'atp', 'isobutanol:e3'], 'massaction', {'k': 'k_uf'})
    rxn14_r = ( ['atp', 'atp', 'atp', 'isobutanol:e3'], ['3atp:isobutanol:e3'], 'massaction', {'k': 'k_ur'})

    rxn15_f = (['isobutanol:e3'], ['isobutanol', 'e3'], 'massaction', {'k': 'k_uf'})
    rxn15_r = (['isobutanol', 'e3'], ['isobutanol:e3'], 'massaction', {'k': 'k_ur'})

    #ATP Leak
    rxn16_f = (["atp", 'e4'], ['atp:e4'], "massaction", {"k":"k_bf"})
    rxn16_r = (['atp:e4'], ["atp", 'e4'],  "massaction", {"k":"k_br"})

    rxn17_f = (['atp:e4'], ["adppi:e4"],  "massaction", {"k":"k_cat"})

    rxn18_f = ( ["adppi:e4"], ['adp', 'pi', 'e4'], "massaction", {"k":"k_uf"})
    rxn18_r = (['adp', 'pi', 'e4'], ["adppi:e4"] , "massaction", {"k":"k_ur"})

    reactions = [rxn1_f, rxn1_r, rxn2_f, rxn2_r, rxn3_f, rxn4_f, rxn4_r, rxn5_f, rxn5_r, rxn6_f, rxn6_r, rxn7_f, rxn7_r,
                rxn8_f, rxn9_f, rxn9_r, rxn10_f, rxn10_r, rxn11_f, rxn11_r, rxn12_f, rxn12_r, rxn13_f,
                rxn14_f, rxn14_r, rxn15_f, rxn15_r, 
                rxn16_f, rxn16_r, rxn17_f, rxn18_f, rxn18_r]

    M = Model(species = species, reactions = reactions, parameters = parameters,
             initial_condition_dict = x_0)
    timepoints = np.linspace(0,t_max,1000)
    df_results = py_simulate_model(timepoints, Model = M, stochastic = False)

    # Model only for the atp leak
    species_atp_leak = ['atp', 'adp', 'pi', 'e4', 'atp:e4', 'adppi:e4']
    reactions_atp_leak = [rxn16_f, rxn16_r, rxn17_f, rxn18_f, rxn18_r]
    M_atp_leak = Model(species = species_atp_leak, reactions = reactions_atp_leak, parameters = parameters,
             initial_condition_dict = x_0_atp_leak)
    df_results_atp_leak = py_simulate_model(timepoints, Model = M_atp_leak, stochastic = False)
    
    return [df_results, df_results_atp_leak]

In [4]:
def model3_df(parameters, x_0, x_0_atp_leak, t_max):
    
    species = ['e1', 'atp', 'e1:2atp', '2atp:glucose:e1', '2adp:f16p:e1', '2adp:e1', 'f16p', '2adp:e1', 'adp', 'pi',
          'e2', 'e2:2adp', '2adp:f16p:e2', '2atp:isobutanol:e2', '2atp:e2', 'isobutanol',
          'e3', 'e2:3adppi', '3adppi:f16p:e3', '3atp:f16p:e3', '3atp:e3']


    rxn1_f = (['e1', 'atp', 'atp'], ['e1:2atp'], 'massaction', {'k': 'k_bf'})
    rxn1_r = (['e1:2atp'], ['e1', 'atp', 'atp'], 'massaction', {'k': 'k_br'})

    rxn2_f = (['glucose', 'e1:2atp'], ['2atp:glucose:e1'], 'massaction', {'k': 'k_bf'})
    rxn2_r = (['2atp:glucose:e1'], ['glucose', 'e1:2atp'], 'massaction', {'k': 'k_br'})

    rxn3_f = (['2atp:glucose:e1'], ['2adp:f16p:e1'], 'massaction', {'k': 'k_cat'})

    rxn4_f = (['2adp:f16p:e1'], ['2adp:e1', 'f16p'], 'massaction', {'k': 'k_uf'})
    rxn4_r = (['2adp:e1', 'f16p'], ['2adp:f16p:e1'], 'massaction', {'k': 'k_ur'})

    rxn5_f = (['2adp:e1'], ['adp', 'adp', 'e1'], 'massaction', {'k': 'k_uf'})
    rxn5_r = (['adp', 'adp', 'e1'], ['2adp:e1'], 'massaction', {'k': 'k_ur'})

    rxn6_f = (['e2', 'adp', 'adp'], ['e2:2adp'], 'massaction', {'k': 'k_bf'})
    rxn6_r = (['e2:2adp'], ['e2', 'adp', 'adp'], 'massaction', {'k': 'k_br'})

    rxn7_f = (['f16p', 'e2:2adp'], ['2adp:f16p:e2'], 'massaction', {'k': 'k_bf'})
    rxn7_r = (['2adp:f16p:e2'], ['f16p', 'e2:2adp'], 'massaction', {'k': 'k_br'})

    rxn8_f = (['2adp:f16p:e2'], ['2atp:isobutanol:e2'], 'massaction', {'k': 'k_cat'})

    rxn9_f = (['2atp:isobutanol:e2'], ['2atp:e2', 'isobutanol'], 'massaction', {'k': 'k_uf'})
    rxn9_r = (['2atp:e2', 'isobutanol'], ['2atp:isobutanol:e2'], 'massaction', {'k': 'k_ur'})

    rxn10_f = (['2atp:e2'], ['atp', 'atp', 'e2'], 'massaction', {'k': 'k_uf'})
    rxn10_r = (['atp', 'atp', 'e2'], ['2atp:e2'], 'massaction', {'k': 'k_ur'})

    rxn11_f = (['e3', 'adp', 'adp', 'adp', 'pi'], ['e3:3adppi'], 'massaction', {'k': 'k_bf'})
    rxn11_r = (['e3:3adppi'], ['e3', 'adp', 'adp', 'adp', 'pi'], 'massaction', {'k': 'k_br'})

    rxn12_f = (['f16p', 'e3:3adppi'], ['3adppi:f16p:e3'], 'massaction', {'k': 'k_bf'})
    rxn12_r = (['3adppi:f16p:e3'], ['f16p', 'e3:3adppi'], 'massaction', {'k': 'k_br'})

    rxn13_f = (['3adppi:f16p:e3'], ['3atp:f16p:e3'], 'massaction', {'k': 'k_cat'})

    rxn14_f = (['3atp:f16p:e3'], ['3atp:e3', 'f16p'], 'massaction', {'k': 'k_uf'})
    rxn14_r = (['3atp:e3', 'f16p'], ['3atp:f16p:e3'], 'massaction', {'k': 'k_ur'})

    rxn15_f = (['3atp:e3'], ['atp', 'atp', 'atp', 'e3'], 'massaction', {'k': 'k_uf'})
    rxn15_r = (['atp', 'atp', 'atp', 'e3'], ['3atp:e3'], 'massaction', {'k': 'k_uf'})


     #ATP Leak
    rxn16_f = (["atp", 'e4'], ['atp:e4'], "massaction", {"k":"k_bf"})
    rxn16_r = (['atp:e4'], ["atp", 'e4'],  "massaction", {"k":"k_br"})

    rxn17_f = (['atp:e4'], ["adppi:e4"],  "massaction", {"k":"k_cat"})

    rxn18_f = ( ["adppi:e4"], ['adp', 'pi', 'e4'], "massaction", {"k":"k_uf"})
    rxn18_r = (['adp', 'pi', 'e4'], ["adppi:e4"] , "massaction", {"k":"k_ur"})

    reactions = [rxn1_f, rxn1_r, rxn2_f, rxn2_r, rxn3_f, rxn4_f, rxn4_r, rxn5_f, rxn5_r, rxn6_f, rxn6_r, rxn7_f, rxn7_r,
                rxn8_f, rxn9_f, rxn9_r, rxn10_f, rxn10_r, rxn11_f, rxn11_r, rxn12_f, rxn12_r, rxn13_f,
                rxn14_f, rxn14_r, rxn15_f, rxn15_r, 
                rxn16_f, rxn16_r, rxn17_f, rxn18_f, rxn18_r]

    M = Model(species = species, reactions = reactions, parameters = parameters,
             initial_condition_dict = x_0)
    timepoints = np.linspace(0,t_max,1000)
    df_results = py_simulate_model(timepoints, Model = M, stochastic = False)

    # Model only for the atp leak
    species_atp_leak = ['atp', 'adp', 'pi', 'e4', 'atp:e4', 'adppi:e4']
    reactions_atp_leak = [rxn16_f, rxn16_r, rxn17_f, rxn18_f, rxn18_r]
    M_atp_leak = Model(species = species_atp_leak, reactions = reactions_atp_leak, parameters = parameters,
             initial_condition_dict = x_0_atp_leak)
    df_results_atp_leak = py_simulate_model(timepoints, Model = M_atp_leak, stochastic = False)
    
    return [df_results, df_results_atp_leak]

Now, let's define all our sliders and focus on the plotting functionality. We will have sliders for all of the rate constants for each model, the initial concentrations of enzyme, glucose, and ATP, and the max time.

In [5]:
k_bf_1_slider = pn.widgets.FloatSlider(name = 'k_bf_1', start = 0, end = 100, step = 0.01, value = 0.378)
k_br_1_slider = pn.widgets.FloatSlider(name = 'k_br_1', start = 0, end = 100, step = 0.00001, value = 0.0378)
k_uf_1_slider = pn.widgets.FloatSlider(name = 'k_uf_1', start = 0, end = 100, step = 0.01, value = 0.04)
k_ur_1_slider = pn.widgets.FloatSlider(name = 'k_ur_1', start = 0, end = 10, step = 0.001, value = 0.004)
k_cat_1_slider = pn.widgets.FloatSlider(name = 'k_cat_1', start = 0, end = 1000, step = 1, value = 200)


k_bf_2_slider = pn.widgets.FloatSlider(name = 'k_bf_2', start = 0, end = 100, step = 0.001, value = 0.378)
k_br_2_slider = pn.widgets.FloatSlider(name = 'k_br_2', start = 0, end = 100, step = 0.00001, value = 0.0378)
k_uf_2_slider = pn.widgets.FloatSlider(name = 'k_uf_2', start = 0, end = 100, step = 0.001, value = 0.04)
k_ur_2_slider = pn.widgets.FloatSlider(name = 'k_ur_2', start = 0, end = 100, step = 0.1000, value = 0.004)
k_cat_2_slider = pn.widgets.FloatSlider(name = 'k_cat_2', start = 0, end = 1000, step = 1, value = 200)


k_bf_3_slider = pn.widgets.FloatSlider(name = 'k_bf_3', start = 0, end = 100, step = 0.01, value = 0.378)
k_br_3_slider = pn.widgets.FloatSlider(name = 'k_br_3', start = 0, end = 100, step = 0.00001, value = 0.0378)
k_uf_3_slider = pn.widgets.FloatSlider(name = 'k_uf_3', start = 0, end = 100, step = 0.01, value = 0.04)
k_ur_3_slider = pn.widgets.FloatSlider(name = 'k_ur_3', start = 0, end = 100, step = 0.001, value = 0.004)
k_cat_3_slider = pn.widgets.FloatSlider(name = 'k_cat_3', start = 0, end = 1000, step = 1, value = 200)

enz_init_slider = pn.widgets.FloatSlider(name = 'initial enzyme', start = 0, end = 1000, step = 1, value = 1.5)
glucose_init_slider = pn.widgets.FloatSlider(name = 'initial glucose', start = 0, end = 1000, step = 1, value = 15)
atp_init_slider = pn.widgets.FloatSlider(name = 'initial atp', start = 0, end = 1000, step = 1, value = 15)

t_max_slider = pn.widgets.FloatSlider(name = 't_max', start = 10, end = 10000, step = 10, value = 1000)

Now, let's make the plotting function.

In [19]:
@pn.depends(enz_init_slider.param.value, glucose_init_slider.param.value, atp_init_slider.param.value,
            k_bf_1_slider.param.value, k_br_1_slider.param.value, k_uf_1_slider.param.value,
            k_ur_1_slider.param.value, k_cat_1_slider.param.value,
           k_bf_2_slider.param.value, k_br_2_slider.param.value, k_uf_2_slider.param.value,
            k_ur_2_slider.param.value, k_cat_2_slider.param.value,
           k_bf_3_slider.param.value, k_br_3_slider.param.value, k_uf_3_slider.param.value,
            k_ur_3_slider.param.value, k_cat_3_slider.param.value, t_max_slider.param.value)

def plot_all(enz_init = 1.5, glucose_init = 15, atp_init = 15,
             k_bf_1 = 0.378, k_br_1 = 0.0378, k_uf_1 = 0.04, k_ur_1 = 0.004, k_cat_1 = 200,
             k_bf_2 = 0.378, k_br_2 = 0.0378, k_uf_2 = 0.04, k_ur_2 = 0.004, k_cat_2 = 200, 
             k_bf_3 = 0.0378, k_br_3 = 0.0378, k_uf_3 = 0.04, k_ur_3 = 0.004, k_cat_3 = 200,
             t_max = 1000):
    
    # Define initial conditions
    x_0 = {
        "glucose":glucose_init,
        "atp":atp_init, 
        'e1': enz_init, 
        'e2':enz_init,
        'e3':enz_init,
        'e4':enz_init
    }

    x_0_atp_leak = { 
        'atp': atp_init,
        'e4':enz_init
    }

    # Define parameter lists here 
    parameters1 = [('k_bf', k_bf_1), ('k_br', k_br_1), ('k_uf', k_uf_1), ('k_ur', k_ur_1), ('k_cat', k_cat_1)]
    parameters2 = [('k_bf', k_bf_2), ('k_br', k_br_2), ('k_uf', k_uf_2), ('k_ur', k_ur_2), ('k_cat', k_cat_2)]
    parameters3 = [('k_bf', k_bf_3), ('k_br', k_br_3), ('k_uf', k_uf_3), ('k_ur', k_ur_3), ('k_cat', k_cat_3)]
    

    timepoints = np.linspace(0,t_max,1000)
    
    # Get data from model one
    model_1_df, model_1_df_atp_leak = model1_df(parameters1, x_0, x_0_atp_leak, t_max)
    
    # First plot glucose and isobutanol
    p1 = bokeh.plotting.figure(width = 300, height = 250, 
                             x_axis_label = 'time',
                             y_axis_label = 'concentration',
                             title = '1 Step Model (massaction)')
    p1.line(timepoints, model_1_df['glucose'], color = 'green', legend_label = 'glucose')
    p1.line(timepoints, model_1_df['isobutanol'], color = 'orange', legend_label = 'isobutanol')
    p1.line(timepoints, model_1_df['f16p'], color = 'purple', legend_label = 'f16p')
    p1.legend.location = 'center_right'

    # Plot ATP and ADP, Pi
    p2 = bokeh.plotting.figure(width = 300, height = 250,
                               x_axis_label = 'time',
                             y_axis_label = 'concentration',
                             title =  f" {parameters1}")
    p2.line(timepoints, model_1_df['atp'], color = 'red', legend_label = 'atp')
    p2.line(timepoints, model_1_df['pi'], color = 'blue', legend_label = 'pi')
    p2.line(timepoints, model_1_df['adp'], color = 'green', legend_label = 'adp')
    p2.legend.location = 'center_right'

    # Plot only ATP leak model
    p3 = bokeh.plotting.figure(width = 300, height = 250,
                              x_axis_label = 'time',
                              y_axis_label = 'concentration',
                              title = 'Only ATP Leak')
    p3.line(timepoints, model_1_df_atp_leak['atp'], legend_label = 'atp', color = 'red')

    
    # Get data from model two
    model_2_df, model_2_df_atp_leak = model2_df(parameters2, x_0, x_0_atp_leak, t_max)
    
      # First plot glucose and isobutanol
    p4 = bokeh.plotting.figure(width = 300, height = 250, 
                             x_axis_label = 'time',
                             y_axis_label = 'concentration (mM)',
                             title = '3 Step Model V1 (massaction)')
    p4.line(timepoints, model_2_df['glucose'], color = 'green', legend_label = 'glucose')
    p4.line(timepoints, model_2_df['isobutanol'], color = 'orange', legend_label = 'isobutanol')
    p4.line(timepoints, model_2_df['f16p'], color = 'purple', legend_label = 'f16p')
    p4.legend.location = 'center_right'

    # Plot ATP and ADP, Pi
    p5 = bokeh.plotting.figure(width = 300, height = 250,
                               x_axis_label = 'time',
                             y_axis_label = 'concentration ',
                              title =  f" {parameters2}")
    p5.line(timepoints, model_2_df['atp'], color = 'red', legend_label = 'atp')
    p5.line(timepoints, model_2_df['pi'], color = 'blue', legend_label = 'pi')
    p5.line(timepoints, model_2_df['adp'], color = 'green', legend_label = 'adp')
    p5.legend.location = 'center_right'

    # Plot only ATP leak model
    p6 = bokeh.plotting.figure(width = 300, height = 250,
                              x_axis_label = 'time ',
                              y_axis_label = 'concentration',
                              title = 'Only ATP Leak')
    p6.line(timepoints, model_2_df_atp_leak['atp'], legend_label = 'atp', color = 'red')
        
    
    # Get data from model three
    model_3_df, model_3_df_atp_leak = model3_df(parameters3, x_0, x_0_atp_leak, t_max)
      # First plot glucose and isobutanol
    p7 = bokeh.plotting.figure(width = 300, height = 250, 
                             x_axis_label = 'time ',
                             y_axis_label = 'concentration',
                             title = '3 Step Model V2 (massaction)')
    p7.line(timepoints, model_3_df['glucose'], color = 'green', legend_label = 'glucose')
    p7.line(timepoints, model_3_df['isobutanol'], color = 'orange', legend_label = 'isobutanol')
    p7.line(timepoints, model_3_df['f16p'], color = 'purple', legend_label = 'f16p')
    p7.legend.location = 'center_right'

    # Plot ATP and ADP, Pi
    p8 = bokeh.plotting.figure(width = 300, height = 250,
                               x_axis_label = 'time (hrs)',
                             y_axis_label = 'concentration',
                              title =  f" {parameters3}")
    p8.line(timepoints, model_3_df['atp'], color = 'red', legend_label = 'atp')
    p8.line(timepoints, model_3_df['pi'], color = 'blue', legend_label = 'pi')
    p8.line(timepoints, model_3_df['adp'], color = 'green', legend_label = 'adp')
    p8.legend.location = 'center_right'

    # Plot only ATP leak model
    p9 = bokeh.plotting.figure(width = 300, height = 250,
                              x_axis_label = 'time',
                              y_axis_label = 'concentration',
                              title = 'Only ATP Leak')
    p9.line(timepoints, model_3_df_atp_leak['atp'], legend_label = 'atp', color = 'red')
    
    
    #return row(p1, p2, p3, p4, p5, p6, p7, p8, p9)
    bokeh.io.show(row(p1, p2, p3))
    bokeh.io.show(row(p4, p5, p6))
    bokeh.io.show(row(p7, p8, p9))
#, row(p4, p5, p6), row(p7, p8, p9))


Now, we format and plot. Ideally, we want to show a 3x3 grid that has p1, p2, p3 on the first row, p4, p5, p6 on the second row, and p7, p8, p9 on the third row.

In [20]:
widgets = pn.Column(
    pn.Spacer(height=10),enz_init_slider, glucose_init_slider, atp_init_slider,
    k_bf_1_slider, k_br_1_slider, k_uf_1_slider, k_ur_1_slider, k_cat_1_slider,
           k_bf_2_slider, k_br_2_slider, k_uf_2_slider, k_ur_2_slider, k_cat_2_slider,
           k_bf_3_slider, k_br_3_slider, k_uf_3_slider, k_ur_3_slider, k_cat_3_slider, t_max_slider,
    width=150,
)

left_column = pn.Column(
    pn.Row(pn.Spacer(width=30)), plot_all
)

# Final layout
pn.Column(left_column, pn.Spacer(width=20), widgets)

Traceback (most recent call last):
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/pyviz_comms/__init__.py", line 315, in _handle_msg
 self._on_msg(msg)
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/panel/viewable.py", line 242, in _on_msg
 doc.unhold()
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/bokeh/document/document.py", line 669, in unhold
 self._trigger_on_change(event)
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/bokeh/document/document.py", line 1132, in _trigger_on_change
 self._with_self_as_curdoc(event.callback_invoker)
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/bokeh/document/document.py", line 1150, in _with_self_as_curdoc
 return f()
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/bokeh/util/callback_manager.py", line 155, in invoke
 callback(attr, old, new)
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/panel/viewable.py", line 494, in _comm_change
 self._process_events({attr: new})
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/panel/viewable.py", line 511, in _process_events
 self.param.set_param(**self._process_property_change(events))
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/param/parameterized.py", line 1365, in set_param
 self_._batch_call_watchers()
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/param/parameterized.py", line 1480, in _batch_call_watchers
 watcher.fn(*events)
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/panel/param.py", line 653, in _replace_pane
 new_object = self.eval(self.object)
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/panel/param.py", line 577, in eval
 return function(*args, **kwargs)
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/param/parameterized.py", line 318, in _depends
 return func(*args,**kw)
 File "<ipython-input-10-ad8beeaa900e>", line 130, in plot_all
 return bokeh.io.show(row (p1, p2, p3),row(p4, p5, p6),row(p7, p8, p9))
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/bokeh/io/showing.py", line 139, in show
 return _show_with_state(obj, state, browser, new, notebook_handle=notebook_handle)
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/bokeh/io/showing.py", line 167, in _show_with_state
 controller = get_browser_controller(browser=browser)
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/bokeh/util/browser.py", line 86, in get_browser_controller
 controller = webbrowser.get(browser)
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/webbrowser.py", line 48, in get
 if '%s' in browser:
TypeError: argument of type 'Row' is not iterable
 


 Traceback (most recent call last):
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/pyviz_comms/__init__.py", line 315, in _handle_msg
 self._on_msg(msg)
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/panel/viewable.py", line 241, in _on_msg
 patch.apply_to_document(doc, comm.id)
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/bokeh/protocol/messages/patch_doc.py", line 100, in apply_to_document
 doc._with_self_as_curdoc(lambda: doc.apply_json_patch(self.content, setter))
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/bokeh/document/document.py", line 1150, in _with_self_as_curdoc
 return f()
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/bokeh/protocol/messages/patch_doc.py", line 100, in <lambda>
 doc._with_self_as_curdoc(lambda: doc.apply_json_patch(self.content, setter))
 File "/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/bokeh/document/document.py", line 411, in apply_json_patch
 patched_obj.set_from_json(a

Column
    [0] Column
        [0] Row
            [0] Spacer(width=30)
        [1] ParamFunction(function)
    [1] Spacer(width=20)
    [2] Column(width=150)
        [0] Spacer(height=10)
        [1] FloatSlider(end=1000, name='initial enzyme', step=1, value=454, value_throttled=454)
        [2] FloatSlider(end=1000, name='initial glucose', step=1, value=15)
        [3] FloatSlider(end=1000, name='initial atp', step=1, value=15)
        [4] FloatSlider(end=100, name='k_bf_1', step=0.01, value=0.378)
        [5] FloatSlider(end=100, name='k_br_1', step=1e-05, value=0.0378)
        [6] FloatSlider(end=100, name='k_uf_1', step=0.01, value=0.04)
        [7] FloatSlider(end=10, name='k_ur_1', step=0.001, value=0.004)
        [8] FloatSlider(end=1000, name='k_cat_1', step=1, value=200)
        [9] FloatSlider(end=100, name='k_bf_2', step=0.001, value=0.378)
        [10] FloatSlider(end=100, name='k_br_2', step=1e-05, value=0.0378)
        [11] FloatSlider(end=100, name='k_uf_2', step=0.001, value=0.04)
        [12] FloatSlider(end=100, name='k_ur_2', value=0.004)
        [13] FloatSlider(end=1000, name='k_cat_2', step=1, value=200)
        [14] FloatSlider(end=100, name='k_bf_3', step=0.01, value=0.378)
        [15] FloatSlider(end=100, name='k_br_3', step=1e-05, value=0.0378)
        [16] FloatSlider(end=100, name='k_uf_3', step=0.01, value=0.04)
        [17] FloatSlider(end=100, name='k_ur_3', step=0.001, value=0.004)
        [18] FloatSlider(end=1000, name='k_cat_3', step=1, value=200)
        [19] FloatSlider(end=10000, name='t_max', start=10, step=10, value=1000)

Note: the k_ur values are reading 0 since they are so small.

In [8]:
%load_ext watermark
%watermark -v -p numpy,bokeh,jupyterlab,bioscrape,panel

CPython 3.7.6
IPython 7.12.0

numpy 1.18.1
bokeh 2.0.2
jupyterlab 1.2.6
bioscrape 0.0.0
panel 0.9.5
